In [3]:
import duckdb
import pandas as pd

# Path to your DuckDB database file
duckdb_file_path = '/workspace/databases/dev.duckdb'

# Use a context manager to handle the DuckDB connection
with duckdb.connect(duckdb_file_path) as con:
    # Query all schemas
    query_schemas = "SELECT schema_name FROM information_schema.schemata"
    schemas = con.execute(query_schemas).fetchdf()
    
    # Print the DataFrame of schemas
    print("Schemas:")
    print(schemas)
    
    # Query all tables in each schema
    query_tables = """
    SELECT table_schema, table_name 
    FROM information_schema.tables 
    WHERE table_schema IN (SELECT schema_name FROM information_schema.schemata)
    """
    tables = con.execute(query_tables).fetchdf()
    
    # Print the DataFrame of tables
    print("\nTables:")
    print(tables)

    # Check if the bronze schema exists
    schema_check_query = "SELECT schema_name FROM information_schema.schemata WHERE schema_name = 'silver'"
    schema_exists = con.execute(schema_check_query).fetchone()
    if schema_exists:
        # Query the table created by dbt
        query = "SELECT * FROM silver.dim_customers LIMIT 5"
        result = con.execute(query).fetchdf()

        # Check if the result is not empty and print the DataFrame
        if not result.empty:
            print(result)
        else:
            print("No records found in the customers table.")
    else:
        print("The 'silver' schema does not exist.")

Schemas:
           schema_name
0               bronze
1                 gold
2   information_schema
3                 main
4           pg_catalog
5               public
6               silver
7   information_schema
8                 main
9           pg_catalog
10  information_schema
11                main
12          pg_catalog

Tables:
   table_schema                    table_name
0          gold               churn_customers
1          gold                churn_features
2          gold        product_usage_features
3          gold  support_interaction_features
4        silver                 dim_customers
5        silver                     dim_dates
6        silver                  dim_products
7        silver            fact_product_usage
8        silver            fact_subscriptions
9        silver     fact_support_interactions
10       bronze                     customers
11       bronze                         dates
12       bronze                      products
13       bronze 

In [ ]:
# Use a context manager to handle the DuckDB connection
with duckdb.connect(duckdb_file_path) as con:
    # Query to delete tables
    query_delete_bronze = "DROP SCHEMA bronze CASCADE;"
    query_delete_silver = "DROP SCHEMA SILVER CASCADE;"
    query_delete_gold = "DROP SCHEMA GOLD CASCADE;"
    con.execute(query_delete_bronze)
    con.execute(query_delete_silver)
    con.execute(query_delete_gold)